## Gestión de ingresos de aerolíneas

¿A qué tarifa se tienen que asignar asientos del avión para maximizar los ingresos?


### Formulación matemática

* $t \in T$: tarifas de asientos
* $s_t$: nº de asientos asignados a cada tarifa
* $p_t$: precio de un asiento en cada tarifa
* $d_t$: tope de demanda de asientos en cada tarifa
* $S$: asientos del avión

\begin{eqnarray*}
\max_{s_t} & \sum_{t \in T} p_t s_t \\
\text{s.t.} & \\
  & s_t \leq d_t & \forall t \in T \\
  & \sum_{t \in T} s_t \leq S \\
  & s_t \geq 0 & \forall t \in T
\end{eqnarray*}



### Datos

Variables|Tarifa regular|Tarifa con descuento
-|-|-
Precio (eur)|400|150
Tope de demanda (asientos)|75|125

* $S$ = 150

In [ ]:
from pandas import Series

In [ ]:
# Nº de asientos
total_seats = 150

# Lista con las tarifas disponibles
tariffs = ["Regular", "Discount"]

print(total_seats, tariffs)

150 ['Regular', 'Discount']


In [ ]:
# Serie con los precios de cada tarifa
prices = Series(index=tariffs, data=[400, 150])
prices

Regular     400
Discount    150
dtype: int64

In [ ]:
# Serie con la demanda de cada tarifa
demands = Series(index=tariffs, data=[75, 125])
demands

Regular      75
Discount    125
dtype: int64

### Resolución con PuLP

In [ ]:
!pip install pulp
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 35.5 MB/s eta 0:00:00


In [ ]:
# Crear problema de maximización
prob = LpProblem("Airfare", LpMaximize)

# Crear variables de asientos de cada tarifa
seats = LpVariable.dicts("Seats", indices=tariffs, lowBound=0, cat=LpInteger)

# Añadir función objetivo al problema
prob += lpSum([prices[t] * seats[t] for t in tariffs])

# Añadir restricciones de no sobrepasar topes de demandas
for t in tariffs:
    prob += seats[t] >= demands[t]

# Añadir restricción de no sobrepasar total de asientos
prob += lpSum([seats[t] for t in tariffs]) == total_seats

# Comprobar que el problema está bien formulado
prob

Airfare:
MAXIMIZE
150*Seats_Discount + 400*Seats_Regular + 0
SUBJECT TO
_C1: Seats_Regular >= 75

_C2: Seats_Discount >= 125

_C3: Seats_Discount + Seats_Regular = 150

VARIABLES
0 <= Seats_Discount Integer
0 <= Seats_Regular Integer

In [ ]:
# Resolver el problema, pintar la solución y ver cuánto da de objetivo
status_num = prob.solve()
status_str = LpStatus[status_num]
if status_str == "Optimal":
  print("Problem solved optimally")
  for v in prob.variables():
      print(f"\tVariable {v.name} = {value(v)}")
  print(f"\tObjective function = {round(value(prob.objective), 2)}")
else:
  print(f"Problem not solved optimally: {status_str}")

Problem not solved optimally: Infeasible


* ¿Qué pasaría si en lugar de maximizar la ganancia buscáramos minimizarla?
* ¿Y si en las restricciones dijéramos que la suma de asientos debe ser exactamente $S$, en vez de menor o igual que $S$?
* ¿Y si suponemos que las demandas que nos dan fueran las mínimas que se deben satisfacer, en lugar de ser las máximas?